In [ ]:
import requests
import json
import pandas as pd
import time
import random
import xmltodict
import os


In [ ]:
conf = open('config.json','r') 
config = json.load(conf)
api_key = config['API_key']
conf.close()
keyw = open('keywords.txt','r')
keywords = keyw.readlines()
keyw.close()
for index in range(len(keywords)):
    keywords[index] = keywords[index].replace('\n','')
print(keywords)
Affiliation = 'New Zealand'
start = 0
count = 25
step = 25
url = "https://api.elsevier.com/content/search/scopus"
headers=  {'X-ELS-APIKey':api_key}
print(url)


In [ ]:
def QueryEdit(url,keyword,Affiliation=None,daterange=None,start = 0,sort = 'citedby-count', count = 25):
    url +='?query={}'.format(keyword)
    if Affiliation!=None:
        url+="+AFFIL({})".format(Affiliation)
    url += "&count={}".format(count)
    url += "&sort={}".format(sort)
    url += "&start={}".format(start)
    return url

In [ ]:
def TidyJsontoDf(web_json,paper_info_short,column):
    try :
        for paper in web_json['search-results']['entry']:
            try:
                title = paper['dc:title']
                autor = paper['dc:creator']
                doi = paper['prism:doi']
                citedby_count = paper['citedby-count']
                affilname = paper['affiliation'][0]['affilname']
                affiliation_city = paper['affiliation'][0]['affiliation-city']
                affiliation_country = paper['affiliation'][0]['affiliation-country']
            except:
                continue
            paper_data = [[title,autor,doi,citedby_count,affilname,affiliation_city,affiliation_country]]
            paper_info = pd.DataFrame(data=paper_data, columns=column)
            paper_info_short = pd.concat([paper_info_short,paper_info],ignore_index=True)
    except:
        pass
    return paper_info_short

In [ ]:
def GetAndTidy(keyword,Affiliation = None,url=url, headers = headers,limit = 5000):
    url = QueryEdit(url,keyword,Affiliation = Affiliation,count = count)
    print(url)
    web = requests.get(url,headers=headers)
    result = web.text
    web_json = json.loads(result)
    print(web_json['search-results']['opensearch:totalResults'])
    epos = int(web_json['search-results']['opensearch:totalResults']) // step +1
    column = ['title','autor','doi','citedby_count','affilname','affiliation-city','affiliation-country']
    paper_info_short = pd.DataFrame(columns=column)
    paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
    for i in range(1,epos+1):
        start = i*25
        if start > limit: break
        url = "https://api.elsevier.com/content/search/scopus"
        url = QueryEdit(url,keyword,Affiliation = Affiliation,start = start,count = count)
        print(url)
        web = requests.get(url,headers=headers)
        if(web.ok):
            result = web.text
            web_json = json.loads(result)
            paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
        # time.sleep(random.random())
    return paper_info_short



### Run one keyword

In [ ]:
keyword = 'geothermal'
url = "https://api.elsevier.com/content/search/scopus"
paper_info = GetAndTidy(keyword,url = url)
paper_info.info()
filepath = 'data/'
paper_info.to_csv(filepath+keyword+'.csv',index=False)

### Run all keywords

In [ ]:
filepath = 'data/'
for keyword in keywords:
    url = "https://api.elsevier.com/content/search/scopus"
    paper_info = GetAndTidy(keyword,url = url)
    paper_info.to_csv(filepath+keyword+'.csv',index=False)

### NZ Data

In [ ]:
filepath = 'data/NZ/'
for keyword in keywords:
    url = "https://api.elsevier.com/content/search/scopus"
    paper_info = GetAndTidy(keyword,Affiliation='New Zealand',limit=500,url = url)
    paper_info.to_csv(filepath+keyword+'_NZ.csv',index=False)

In [ ]:
# clean nz data
filepath = 'data/NZ/'
filelist = os.listdir(filepath)
print(filelist)
for file in filelist:
    df = pd.read_csv(filepath + file)
    selected_data = df.query(" `affiliation-country` == 'New Zealand' ")
    selected_data.to_csv(filepath+file,index=False)



### TEST

In [ ]:
url = "https://api.elsevier.com/content/search/scopus"
keyword = 'additive manufacturing'
url = QueryEdit(url,keyword,Affiliation = None,count = 25)
print(url)
web = requests.get(url,headers=headers)
result = web.text
web_json = json.loads(result)
print(web_json)